In [1]:
#Import

#pandas
import pandas as pd
from pandas import Series,DataFrame

#numpy
import numpy as np

In [2]:
#路径
path='C:\Users\Wvv\Datacastle_subsidy'

#借书记录
borrow_train_name=['id','borrow_date','book_name','book_num']
borrow_train=pd.read_table(path+'/train/borrow_train.txt',sep=',',header=None,names=borrow_train_name)

#删除重复行并浏览前几行数据
borrow_train=borrow_train.drop_duplicates()
borrow_train.head()

,id,borrow_date,book_name,book_num
0,9708,2014/2/25,我的英语日记 wo de ying yu ri ji / (韩)南银英著 (韩)卢炫廷插图,H315 502
1,6956,2013/10/27,解读联想思维 jie du lian xiang si wei : 联想教父柳传志,K825.38=76 547
2,23939,2015/3/8,电路分析 dian lu fen xi 刘健主编,TM133-43 56
3,22047,2014/12/29,现代物流学 xian dai wu liu xue = Contemporary logis...,F252 161
4,9076,2014/3/28,"公司法 gong si fa = = Corporation law / 范健, 王建文著 eng",NaN


In [3]:
#删除无用列

borrow_train.drop(['book_name','book_num'],axis=1,inplace=True)

In [4]:
#定义time转化函数

from datetime import datetime
def time_transform(x):
    return datetime.strptime(x,'%Y/%m/%d')

In [5]:
#转换字符串型的time成时间戳格式
borrow_train['borrow_date']=borrow_train['borrow_date'].apply(time_transform)

In [6]:
#时间格式转换

borrow_train['month']=borrow_train['borrow_date'].apply(lambda x:x.strftime('%m'))
borrow_train['weekday']=borrow_train['borrow_date'].apply(lambda x:x.strftime('%A'))

In [7]:
#month
b_month=borrow_train.groupby('month')['month'].count()
#对month进行one-hot编码

month_dummies=pd.get_dummies(borrow_train['month'])
month_dummies.columns=['borrow_month'+x for x in list(b_month.index)]

#连接两表

borrow_train=borrow_train.join(month_dummies)

In [8]:
#weekday（周末与非周末）

def get_week(week):
    a=['Monday','Tuesday','Wednesday','Thursday','Friday']
    #b=['Saturday','Sunday']
    if week in a:
        return 'wd'
    else:
        return 'vd'

borrow_train['weekday']=borrow_train['weekday'].apply(get_week)

week_dummies=pd.get_dummies(borrow_train['weekday'])
week_dummies.columns=['borrow_resting_day','borrow_working_day']

borrow_train=borrow_train.join(week_dummies)

In [9]:
#根据id聚合求解各个特征的产生的总数

count_columns=list(borrow_train.columns)[4:]

#count_columns
count_pivot=borrow_train.pivot_table(index='id',values=count_columns,aggfunc=np.count_nonzero)

In [10]:
#每个id借书的总次数

borrow_train_feature=DataFrame(borrow_train.groupby('id')['id'].count())
borrow_train_feature.columns=['borrow_sum']

#连接表
borrow_train_feature=borrow_train_feature.join(count_pivot)

In [11]:
#周末与非周末借书占比

borrow_week_rate=borrow_train.pivot_table(index='id',values=['borrow_resting_day','borrow_working_day'],aggfunc=np.mean)
borrow_week_rate.columns=['borrow_resting_day_rate','borrow_working_day_rate']

#连接表
borrow_train_feature=borrow_train_feature.join(borrow_week_rate)

In [12]:
#每个id活跃天数

borrow_day=DataFrame(borrow_train.groupby(['id','borrow_date'])['id'].count())
borrow_day.to_csv('borrow_input/borrow_train_day.csv',index=True)
borrow_day_count=pd.read_csv('borrow_input/borrow_train_day.csv')
borrow_day_count=DataFrame(borrow_day_count.groupby('id')['id.1'].count())
borrow_day_count.columns=['borrow_day_sum']

#连接表
borrow_train_feature=borrow_train_feature.join(borrow_day_count)

In [13]:
#每天平均借书量

borrow_train_feature['borrow_day_per']=borrow_train_feature['borrow_sum'].divide(borrow_train_feature['borrow_day_sum'],fill_value=0)

In [14]:
#导出数据

borrow_train_feature.to_csv('borrow_input/borrow_train_feture_f.csv',index='id')

In [15]:
borrow_train_feature.head()

,borrow_sum,borrow_month01,borrow_month02,borrow_month03,borrow_month04,borrow_month05,borrow_month06,borrow_month07,borrow_month08,borrow_month09,borrow_month10,borrow_month11,borrow_month12,borrow_resting_day,borrow_working_day,borrow_resting_day_rate,borrow_working_day_rate,borrow_day_sum,borrow_day_per
id,,,,,,,,,,,,,,,,,,,
11,2,0,0,0,0,2,0,0,0,0,0,0,0,0,2,0.0,1.0,1,2.000000
20,5,0,0,0,0,2,0,0,0,1,2,0,0,1,4,0.2,0.8,3,1.666667
22,3,0,0,0,0,0,0,0,0,0,0,3,0,0,3,0.0,1.0,1,3.000000
23,7,0,0,5,0,2,0,0,0,0,0,0,0,7,0,1.0,0.0,2,3.500000
28,4,1,3,0,0,0,0,0,0,0,0,0,0,0,4,0.0,1.0,4,1.000000


In [16]:
borrow_train.head()

,id,borrow_date,month,weekday,borrow_month01,borrow_month02,borrow_month03,borrow_month04,borrow_month05,borrow_month06,borrow_month07,borrow_month08,borrow_month09,borrow_month10,borrow_month11,borrow_month12,borrow_resting_day,borrow_working_day
0,9708,2014-02-25,02,wd,0,1,0,0,0,0,0,0,0,0,0,0,0,1
1,6956,2013-10-27,10,vd,0,0,0,0,0,0,0,0,0,1,0,0,1,0
2,23939,2015-03-08,03,vd,0,0,1,0,0,0,0,0,0,0,0,0,1,0
3,22047,2014-12-29,12,wd,0,0,0,0,0,0,0,0,0,0,0,1,0,1
4,9076,2014-03-28,03,wd,0,0,1,0,0,0,0,0,0,0,0,0,0,1


In [17]:
count_pivot.head()

,borrow_month01,borrow_month02,borrow_month03,borrow_month04,borrow_month05,borrow_month06,borrow_month07,borrow_month08,borrow_month09,borrow_month10,borrow_month11,borrow_month12,borrow_resting_day,borrow_working_day
id,,,,,,,,,,,,,,
11,0,0,0,0,2,0,0,0,0,0,0,0,0,2
20,0,0,0,0,2,0,0,0,1,2,0,0,1,4
22,0,0,0,0,0,0,0,0,0,0,3,0,0,3
23,0,0,5,0,2,0,0,0,0,0,0,0,7,0
28,1,3,0,0,0,0,0,0,0,0,0,0,0,4


In [18]:
borrow_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136705 entries, 0 to 227325
Data columns (total 18 columns):
id                    136705 non-null int64
borrow_date           136705 non-null datetime64[ns]
month                 136705 non-null object
weekday               136705 non-null object
borrow_month01        136705 non-null uint8
borrow_month02        136705 non-null uint8
borrow_month03        136705 non-null uint8
borrow_month04        136705 non-null uint8
borrow_month05        136705 non-null uint8
borrow_month06        136705 non-null uint8
borrow_month07        136705 non-null uint8
borrow_month08        136705 non-null uint8
borrow_month09        136705 non-null uint8
borrow_month10        136705 non-null uint8
borrow_month11        136705 non-null uint8
borrow_month12        136705 non-null uint8
borrow_resting_day    136705 non-null uint8
borrow_working_day    136705 non-null uint8
dtypes: datetime64[ns](1), int64(1), object(2), uint8(14)
memory usage: 7.0+ MB


In [19]:
len(count_pivot)

7181

In [20]:
borrow_train_feature.head()

,borrow_sum,borrow_month01,borrow_month02,borrow_month03,borrow_month04,borrow_month05,borrow_month06,borrow_month07,borrow_month08,borrow_month09,borrow_month10,borrow_month11,borrow_month12,borrow_resting_day,borrow_working_day,borrow_resting_day_rate,borrow_working_day_rate,borrow_day_sum,borrow_day_per
id,,,,,,,,,,,,,,,,,,,
11,2,0,0,0,0,2,0,0,0,0,0,0,0,0,2,0.0,1.0,1,2.000000
20,5,0,0,0,0,2,0,0,0,1,2,0,0,1,4,0.2,0.8,3,1.666667
22,3,0,0,0,0,0,0,0,0,0,0,3,0,0,3,0.0,1.0,1,3.000000
23,7,0,0,5,0,2,0,0,0,0,0,0,0,7,0,1.0,0.0,2,3.500000
28,4,1,3,0,0,0,0,0,0,0,0,0,0,0,4,0.0,1.0,4,1.000000


In [21]:
borrow_train_feature.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7181 entries, 11 to 32643
Data columns (total 19 columns):
borrow_sum                 7181 non-null int64
borrow_month01             7181 non-null uint8
borrow_month02             7181 non-null uint8
borrow_month03             7181 non-null uint8
borrow_month04             7181 non-null uint8
borrow_month05             7181 non-null uint8
borrow_month06             7181 non-null uint8
borrow_month07             7181 non-null uint8
borrow_month08             7181 non-null uint8
borrow_month09             7181 non-null uint8
borrow_month10             7181 non-null uint8
borrow_month11             7181 non-null uint8
borrow_month12             7181 non-null uint8
borrow_resting_day         7181 non-null uint8
borrow_working_day         7181 non-null uint8
borrow_resting_day_rate    7181 non-null float64
borrow_working_day_rate    7181 non-null float64
borrow_day_sum             7181 non-null int64
borrow_day_per             7181 non-null 